In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools

import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from brew.base import Ensemble, EnsembleClassifier
from brew.stacking.stacker import EnsembleStack, EnsembleStackClassifier
from brew.combination.combiner import Combiner
from brew.generation import Bagging


from mlxtend.data import iris_data
from mlxtend.evaluate import plot_decision_regions

In [6]:
X, y = iris_data()
X = X[:,[0, 2]]

In [9]:
# Initializing ensemble of stump decision trees
clf = DecisionTreeClassifier(max_depth=1)
bag = Bagging(base_classifier=clf, n_classifiers=20)
bag.fit(X, y)

ensemble = bag.ensemble

In [10]:
from brew.selection.dynamic.ola import OLA
from brew.selection.dynamic.lca import LCA
from brew.selection.dynamic.probabilistic import APriori, APosteriori
from brew.selection.dynamic.knora import KNORA_UNION, KNORA_ELIMINATE

In [ ]:

clf1 = EnsembleClassifier(ensemble=ensemble, selector=OLA(X, y), combiner=Combiner('mean'))
clf2 = EnsembleClassifier(ensemble=ensemble, selector=LCA(X, y), combiner=Combiner('mean'))
clf3 = EnsembleClassifier(ensemble=ensemble, selector=APriori(X, y), combiner=Combiner('mean'))
clf4 = EnsembleClassifier(ensemble=ensemble, selector=APosteriori(X, y), combiner=Combiner('mean'))
clf5 = EnsembleClassifier(ensemble=ensemble, selector=KNORA_ELIMINATE(X, y), combiner=Combiner('mean'))
clf6 = EnsembleClassifier(ensemble=ensemble, selector=KNORA_UNION(X, y), combiner=Combiner('mean'))





# Creating Ensemble
ensemble = Ensemble([clf1, clf2, clf3])
eclf = EnsembleClassifier(ensemble=ensemble, combiner=Combiner('mean'))

# Creating Stacking
layer_1 = Ensemble([clf1, clf2, clf3])
layer_2 = Ensemble([sklearn.clone(clf1)])

stack = EnsembleStack(cv=3)

stack.add_layer(layer_1)
stack.add_layer(layer_2)

sclf = EnsembleStackClassifier(stack)

clf_list = [clf1, clf2, clf3, eclf, sclf]
lbl_list = ['Logistic Regression', 'Random Forest', 'RBF kernel SVM', 'Ensemble', 'Stacking']

# Loading some example data


# Plotting Decision Regions
gs = gridspec.GridSpec(2, 3)
fig = plt.figure(figsize=(10, 8))

itt = itertools.product([0, 1, 2], repeat=2)

for clf, lab, grd in zip(clf_list, lbl_list, itt):
    clf.fit(X, y)
    ax = plt.subplot(gs[grd[0], grd[1]])
    fig = plot_decision_regions(X=X, y=y, clf=clf, legend=2)
    plt.title(lab)
plt.show()